# Projet réalisé en début de Master

In [ ]:
import pandas as pd

# read the csv-formatted data file into a pandas dataframe
df=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
# get shape of data frame
print('Shape (n_rows,n_columns) of dataframe:',df.shape)
# print top 5 rows (default) of the data frame
df.head()

# Classification en utilisant un modèle Knn

In [ ]:
import pandas as pd
import numpy as np
import math


def spitData(df, test):
    nbLignes = df.shape[0]
    melange = df.sample(nbLignes)
    train = melange.iloc[:int(nbLignes *test)]
    test = melange.iloc[int(nbLignes *test):]
    return train, test

def replaceZero(data):
    colonnes=['Glucose', 'BloodPressure', 'SkinThickness','BMI']
    for c in colonnes:
        moyenne = df[data>0][[c]].mean()
        df[c] = np.where(df[[c]]>0,df[[c]],moyenne)

def similarity(element1,element2) :
    distance = 0
    for i in range(0,element2.count()):
        distance += math.pow((element1.iloc[i]-element2[i]), 2)
    return math.sqrt(distance)

def neighbour(test,train) :
    distances = []
    length = len(test)-1
    for x in range(len(train)):
        dist = similarity(test,train.iloc[x])
        distances.append((train.iloc[x], dist))
    return distances[:5]

def result(test,train) :
    diabete, noDiabete = 0, 0
    neigh = neighbour(test, train)
    for i in neigh:
        if i[0][7] == 1:
            diabete += 1
        else:
            noDiabete += 1
    return 1 if diabete>noDiabete else 0


def accuracy(test,train) :
    correct = 0
    for x in range(len(test)):
        if result(test.iloc[x],train)==int(test.iloc[x]["Outcome"]):
            correct += 1
    print((correct/float(len(test))) * 100.0)

def knn() :
    df.drop('Insulin',axis=1,inplace=True)
    train, test = spitData(df,0.3)
    replaceZero(train)
    replaceZero(test)
    accuracy(test,train)
knn()

# Classification en utilisant un modèle à base d'arbres de décision

In [ ]:
# Retrait physique de la colonne _Insulin_ (axis=1 signifie par son nom)
donnees = df
donnees.drop('Insulin',axis=1,inplace=True)
# où l'on voit bien que l'objet 'donnees' est bien le même objet que 'df' !!!
df.columns

# Avec la fonction train_test_split de sklearn on peut facilement diviser notre dataframe
# en ensembles train/test 
import sklearn
from sklearn.model_selection import train_test_split
# 0,3 pour préciser que 30% seront pour l'ensemble de test et 70% pour l'entrainement
train, test = train_test_split(donnees, test_size = 0.3, random_state = 0) 

# Visualisaton des ensembles
train.describe()

# La bibliothèque numpy offre plusieurs fonctions très utiles. Par exemple, elle nous permet de créer une nouvelle colonne dans notre
# dataframe si une condition est satisfaite
import numpy as np

def imputeColumns(dataset):
    """ Pour chacune des colonnes du dataset,
        mise à jour des valeurs à zero par la moyenne de ses valeurs non nulles.
    """
    # liste des colonnes qui seront traitées
    columnsToImpute=['Glucose', 'BloodPressure', 'SkinThickness','BMI']

    for c in columnsToImpute:
        avgOfCol=dataset[dataset[c]>0][[c]].mean()
        dataset[c+'_imputed']=np.where(dataset[[c]]!=0,dataset[[c]],avgOfCol)
#        dataset[c]=np.where(dataset[[c]]!=0,dataset[[c]],avgOfCol)

imputeColumns(train)
imputeColumns(test)
# check that we've imputed the 0 values  
train[['Glucose','Glucose_imputed']].head()

# Construction des sous-ensembles Entrée (X) et Sortie (Y) 
X_train = train[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness','BMI', 'DiabetesPedigreeFunction', 'Age']]
Y_train = train[['Outcome']]
X_test = test[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
Y_test = test[['Outcome']]


from sklearn import tree

# Create the classifier
mon_arbre_de_decision = tree.DecisionTreeClassifier(random_state = 0)

# Train the classifier on the training set
mon_arbre_de_decision.fit(X_train, Y_train)

# Evaluate the classifier on the testing set using classification accuracy
mon_arbre_de_decision.score(X_test, Y_test)


#import graphviz
#resultat_visuel = tree.export_graphviz(mon_arbre_de_decision, out_file=None)
#graph = graphviz.Source(resultat_visuel)
#graph
                             
#dot_data = export_graphviz(mon_arbre_de_decision, out_file=None)

# Visualisation des données

In [ ]:
# get a plotting library
import matplotlib.pyplot as plt
# make it interactive in the notebook (Only needed for that librairy)
%matplotlib inline 

# plot Glucose vs BloodPressure and color points according to Outcome
plt.figure()
plt.scatter(df[df.Outcome==1].BloodPressure,df[df.Outcome==1].Glucose,label='Diabete',color='r', marker="o",s=5)
plt.scatter(df[df.Outcome==0].BloodPressure,df[df.Outcome==0].Glucose,label='No Diabete',color='b',marker="x",s=5)
plt.legend()
plt.xlabel('BloodPressure')
plt.ylabel('Glucose')

# superposition des histogrammes de l'Age en fonction du diabète
plt.figure()
plt.hist(df[df.Outcome==1]['Age'],bins=15,label='Diabetes',color='r',alpha=0.2)
plt.hist(df[df.Outcome==0]['Age'],bins=15,label='No Diabetes',color='b',alpha=0.2)
plt.xlabel('Age')
plt.ylabel('Frequence')
plt.legend()
plt.show()

In [ ]:
# Pour chacun des attributs
for c in df.columns:
    plt.figure()               # On construit un nouveau dessin
    plt.hist(df[c],bins=15)    # On regroupe les valeurs de la colonne 'c' par plage de largeur 15
    plt.xlabel(c)              # On donne le nom de l'attribut pour l'abscisse
    plt.ylabel('Fréquence')    # On nomme l'axe des ordonnées "Fréquence"
    plt.show()             